<h1>Содержание<span class="tocSkip"></span></h1>

* [1  Загрузка и подготовка данных](#Загрузка-и-подготовка-данных)
* [1.1 Импорт и предобработка данных](#Импорт-и-предобработка-данных)
* [1.2  Подготовка данных](#Подготовка-данных)
* [2  Обучение и проверка модели](#Обучение-и-проверка-модели)
* [3  Подготовка к расчёту прибыли](#Подготовка-к-расчёту-прибыли)
* [3.1  Нахождение точки безубыточности скважины по объему запасов](#Нахождение-точки-безубыточности-скважины-по-объему-запасов)
* [3.2  Написание функции для расчета прибыли](#Написание-функции-для-расчета-прибыли)
* [4  Расчёт прибыли и рисков](#Расчёт-прибыли-и-рисков)
* [5  Чек-лист готовности проекта](#Чек-лист-готовности-проекта)
* [Общий вывод](#Общий-вывод)

# Самостоятельный проект: «Выбор локации для скважины»

**Цель самостоятельного проекта:**

Построить для компании «ГлавРосГосНефть» модель машинного обучения, которая определит регион, где добыча принесёт наибольшую прибыль.

**Источники информации:**
Данные геологоразведки трёх регионов с идентификаторами скважин id, признаками точек f0, f1 и f2, а также данными по объекмам запасов product в тысячах баррелей.

Для потроения модели необходимо решить следующие **задачи**:
1. J,exbnm модели для каждого из трех регионов (алгоритм LinearRegression);
2. Оценить модели с помощью метрик регрессии;
3. Произвести расчет точки безубыточности по объему для одной скважины;
4. Написать функцию для расчета прибыли;
5. С помощью техники Bootstrap проанализировать возможную прибыль и риски для каждого региона.

## Загрузка и подготовка данных

### Импорт и предобработка данных

Загружаю библиотеки и модули, с помощью которых я буду выполнять проект. Далее импортируем DataSet'ы, в которых хранятся интересующие данные (импорт производится в переменные 'dataset_0', 'dataset_1', 'dataset_2'):

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats as st

In [2]:
region0 = pd.read_csv('geo_data_0.csv')

In [3]:
region1 = pd.read_csv('geo_data_1.csv')

In [4]:
region2 = pd.read_csv('geo_data_2.csv')

Проверяю размеры переменной **region_0** полученных переменных с помощью метода .info(), а также наличие строк - дубликатов с помощью методов duolicated() и sum(). Для ознакомления вывел через метод .head() первые пять значений полученных DataFrame'ов:

In [5]:
region0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
region0.duplicated().sum()

0

In [7]:
region0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


В переменной все типы столбцов соответствуют их содержанию, дублирующиеся строки отсутствуют.

Аналогичным образом проверяю переменную **region_1**:

In [8]:
region1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
region1.duplicated().sum()

0

In [10]:
region1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


В переменной region_1 также все типы столбцов соответствуют их содержанию, дублирующиеся строки отсутствуют.

Проверяю переменную **region_2**:

In [11]:
region2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [12]:
region2.duplicated().sum()

0

In [13]:
region2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Краткий вывод:**

Все три переменные имеют размер 10 000 строк на 5 столбцов. Типы столбцов соответствует их содержанию, отсутствуют дублирующиеся и пустые строки.

### Подготовка данных

В данном разделе произвел разделение переменных на обучающий и валидационный наборы данных в разбивке по регионам. Разделение на наборы данных произвел с помощью функции .train_test_split. Так как масштабы в признаках 'f0', 'f1', 'f2' в полученных DataFrame'ах примерно сопоставимы, решил не производить масштабирование.

**Разделение данных на наборы**

При разделении наборов данных разбил исходные DataSet'ы на обучающую и валидационную выборки по паритету 0.75 / 0.25:

In [14]:
train_0, valid_0, train_1, valid_1, train_2, valid_2 = train_test_split(
    region0, region1, region2, test_size = 0.25)

Проверяю размеры новых наборов данных:

Для **region_0**:

In [15]:
print(train_0.shape)
print(valid_0.shape)

(75000, 5)
(25000, 5)


Для **region_1**:

In [16]:
print(train_1.shape)
print(valid_1.shape)

(75000, 5)
(25000, 5)


Для region_2:

In [17]:
print(train_2.shape)
print(valid_2.shape)

(75000, 5)
(25000, 5)


Разделение на наборы данных произошло по схеме '75 % / 25 %'.

Далее разбиваем наборы данных на признаки и целевые признаки (при формировании наборов с признаками помимо целевого признака также удалил признак 'id', так как он не нужен для обучения модели):

**Признаки**:

In [18]:
features_train_0 = train_0.drop(['id', 'product'], axis = 1)
features_train_1 = train_1.drop(['id', 'product'], axis = 1)
features_train_2 = train_2.drop(['id', 'product'], axis = 1)

features_valid_0 = valid_0.drop(['id', 'product'], axis = 1)
features_valid_1 = valid_1.drop(['id', 'product'], axis = 1)
features_valid_2 = valid_2.drop(['id', 'product'], axis = 1)

**Целевые признаки**

In [19]:
target_train_0 = train_0['product']
target_train_1 = train_1['product']
target_train_2 = train_2['product']


target_valid_0 = valid_0['product']
target_valid_1 = valid_1['product']
target_valid_2 = valid_2['product']

Данные по скважинам в трех регионах импортированы, обучающий и валидационный наборы данных сформированы. Переходим к обучению моделей для регионов.

## Обучение и проверка модели

Ниже проведем обучение модели с помощью алгоритма LinearRegression() для каждого региона, после чего посчитаем среднее значение прогнозированных запасов скважин (метод .mean()) по регионам, а также перепроверим модели с помощью метрик root mean squared error, r2 и mean absolute error (соответственно через функции mean_squared_error, от которой получим корень, r2_score и mean_absolute_error):

**Регион 0**

Обучение модели и получение прогнозных данных на валидационном наборе для Региона 0:

In [20]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0)

In [21]:
predicted_valid_0.mean()

92.53009385867597

Расчитываю mse и rmse для Региона 0:

In [22]:
mse = mean_squared_error(target_valid_0, predicted_valid_0)

In [23]:
mse

1404.5802061605139

In [24]:
rmse = mse ** 0.5

In [25]:
rmse

37.47772946911957

Считаю коэффициент детерминации (мера r2):

In [26]:
r2_reg0 = r2_score(target_valid_0, predicted_valid_0)
r2_reg0

0.27962601129430953

Считаю коэффициент mae:

In [27]:
mae_reg0 = mean_absolute_error(target_valid_0, predicted_valid_0)
mae_reg0

30.845232656375774

**Регион 1**

Обучение модели и получение прогнозных данных на валидационном наборе для Региона 1:

In [28]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)

In [29]:
predicted_valid_1.mean()

68.45931083646666

Расчитываю mse и rmse для Региона 1:

In [30]:
mse = mean_squared_error(target_valid_1, predicted_valid_1)

In [31]:
mse

0.8004145668182078

In [32]:
rmse = mse ** 0.5

In [33]:
rmse

0.8946589108806818

Считаю коэффициент детерминации (мера r2):

In [34]:
r2_reg1 = r2_score(target_valid_1, predicted_valid_1)
r2_reg1

0.9996204479839186

Считаю коэффициент mae:

In [35]:
mae_reg1 = mean_absolute_error(target_valid_1, predicted_valid_1)
mae_reg1

0.7194446802716074

**Регион 2**

Обучение модели и получение прогнозных данных на валидационном наборе для Региона 2:

In [36]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)

In [37]:
predicted_valid_2.mean()

95.13237002558567

Расчитываю mse и rmse для Региона 2:

In [38]:
mse = mean_squared_error(target_valid_2, predicted_valid_2)

In [39]:
mse

1609.2103965794283

In [40]:
rmse = mse ** 0.5

In [41]:
rmse

40.11496474608233

Считаю коэффициент детерминации (мера r2):

In [42]:
r2_reg2 = r2_score(target_valid_2, predicted_valid_2)
r2_reg2

0.19926904606331697

Считаю коэффициент mae:

In [43]:
mae_reg2 = mean_absolute_error(target_valid_2, predicted_valid_2)
mae_reg2

32.932770507865364

**Краткий вывод:**

Как отмечено выше, в данном разделе были расчитаны предиктивные модели для прогнозирования запасов по регионам 0, 1 и 2. Для построения модели использовался алгоритм линейной регрессии LinearRegression() - исходя из постановки задачи, после чего был рассчитан средний объем запасов по скважинам. Наибольшие средние запасы нефти на скважину получены в регионе 2 (95 тысяч баррелей), вторым по ровню средних запасов идет регион 0 (92 тысяч баррелей), а третьим регион 1 (39 тысяч баррелей).

При этом оценка моделей с помощью метрик регрессии показала, что наилучшие характеристики rmse, r2 и mae у региона 1 (мера r2 составила почти 1х(!), а mae лишь 0,71х).

## Подготовка к расчёту прибыли

### Нахождение точки безубыточности скважины по объему запасов

В данном разделе произведем расчет точки безубыточности по объему сырья для разработки новой скважины, далее сравним полученный объём сырья со средним запасом в каждом регионе.

Создаю ниже следюущие константы:

**NUMBER_OF_WELLS** - количество скважин (так как в каждом регионе будут отобраны только 200 скважин для разаботки, переменной будет присвоено значение 200);

**COST_PER_REGION** - бюджет на разработку скважин в регионе (согласно поставновки задачи он составляет 10 млрд. рос. рублей);

**COST_PER_WELL** - бюджет на разработку одной скважины (найдем ее, исходя из значений переменных выше);

**PRICE_PER_UNIT** - доход с единцы продукции (450 рос. рублей с 1 барелля, соответственно, 450 тыс. рос. рублей - так как единца измерений целевого признака тысяча бареллей).


In [44]:
NUMBER_OF_WELLS = 200
COST_PER_REGION = 10000000000
PRICE_PER_UNIT = 450000

Значения констанам NUMBER_OF_WELLS, COST_PER_REGION и PRICE_PER_UNIT присвоены, рассчитаем переменную COST_PER_WELL:

In [45]:
COST_PER_WELL = COST_PER_REGION / NUMBER_OF_WELLS
COST_PER_WELL

50000000.0

Соответственно, затраты на 1 скважину составляют 50 млн. рос. рублей. Далее рассчитаем каков должен быть объем скважины для того, чтобы по прибыль по ней была на уровне точки безубыточности (значение сохраню в константе **BREAK_EVEN_VOLUME**, для удобства округлим его до 3 знаков после запятой):

In [46]:
BREAK_EVEN_VOLUME = round(COST_PER_WELL / PRICE_PER_UNIT, 3)
BREAK_EVEN_VOLUME

111.111

**Краткий вывод:**

Приведенный выше расчет показывает, что объем 1 скажины для достижения точки безубыточности должен быть не менее 111.111 единиц продукции (она измеряется в тысячах бареллей). Средний объем запасов в каждом регионе меньше уровня точки безубыточности для 1 скважины.

### Написание функции для расчета прибыли

Так как объем запасов скважины для добычи на уровне точки безубыточности рассчитан, далее создаем функцию **profit_calculation** для расчёта прибыли по выбранным скважинам и предсказаниям модели. Функция производит расчет следующим образом - в виде аргументов в нее подаются результат предсказаний и целевые признаки. Далее данные предсказаний сортируются по убыванию (метод .sort_value()) и сохраняются в переменную predictions_sorted. После в переменной selected сохраняются целевые признаки первых 200 строк из набора target (для их поиска используются соответствующие индексы из набора predictions_sorted).

Далее в переменной profit нахожу выручку от предлагаемых скважин (для этого в начале складываю объем скважин через метод .sum(), потом умножаю его на константу цены PRICE_PER_UNIT) и вычитаю из нее стоимость разработки скважин в регионе (константа COST_PER_REGION). Именно значение переменной profit возвращает функция:

In [47]:
def profit_calculation(predictions, target):
    predictions_sorted = predictions.sort_values(ascending = False)
    selected = target[predictions_sorted.index][:200]
    profit = selected.sum() * PRICE_PER_UNIT - COST_PER_REGION
    return profit

**Краткий вывод:**

Функция для расчет прибыли создана, в следующем разделе произведем с ее помощью расчет рисков и прибыли для каждого региона. 

## Расчёт прибыли и рисков 

**Регион 0:**

Находим распределение прибыли в регионе 0 с помощью техники Bootstrap с 1000 выборок. В начале создал переменные features0 и target0, куда сохранил все признаки и целевые признаки для региона 0.

Далее в переменной predicted_0 сохранил все прогнозируемые значения по признакам (модель для региона 0 уже обучил ранее - model_0).

Далее через цикл с оператором перебора range (в котором указано значение 1000 - количество выборок) создал список прибылей для получившихся выборок (использовал ранее созданную функцию profit_calculation, куда в качестве аргументов передавал выборки по 500 элементов с прогнозами и целевыми признаками из переменных target0 и predicted_0). Для получения выборок использовал метод .sample, где в качестве n передал значение 500 (размер выборки), в качестве параметра random_state передал переменную state (для нее создал объект random_state из модуля numpy.random), в качестве replace указал True (чтобы обеспечить выбор элементов с возвращением).

Значение прибыли по каждой выборке добавляется в список values. Далее в переменную avr_profit0 считаю среднее значение из values по региону 0:

In [48]:
features_0 = region0.drop(['id', 'product'], axis = 1)
target0 = region0['product']
predicted_0 = model_0.predict(features_0)
predicted_0 = pd.Series(predicted_0)

In [49]:
state = np.random.RandomState(12345)

In [50]:
values = []
for i in range(1000):
    target_subsample = target0.sample(n = 500, replace=True, random_state=state)
    probs_subsample = predicted_0[target_subsample.index]
    values.append(profit_calculation(probs_subsample, target_subsample))

values = pd.Series(values)
avr_profit0 = values.mean()
avr_profit0

431285313.034662

Средняя прибыль для региона 0 составила 430 млн. рос. рублей. Ниже считаю доверительный интервал с помощью метода .quantile(), примененного к values:

In [51]:
lower = values.quantile(0.025)
upper = values.quantile(0.975)
print('2.5% квантиль {: .10}'.format(lower))
print('97.5% квантиль {: .10}'.format(upper))

2.5% квантиль -60694408.75
97.5% квантиль  984211546.3


Диапазон доверительного интервала составил от '-62' млн. рос. рублей до 984 млн. рос. рублей. Далее считаем вероятность убытка через метод mean(), примененный к формуле (values < 0):

In [52]:
risks0 = (values < 0).mean()
risks0

0.05

Вероятность убытка составила 5 %.

**Регион 1:**

Считаем значение прибыли для региона 1:

In [53]:
features_1 = region1.drop(['id', 'product'], axis = 1)
target1 = region1['product']
predicted_1 = model_1.predict(features_1)
predicted_1 = pd.Series(predicted_1)

In [54]:
values = []
for i in range(1000):
    target_subsample = target1.sample(n = 500, replace=True, random_state=state)
    probs_subsample = predicted_1[target_subsample.index]
    values.append(profit_calculation(probs_subsample, target_subsample))

values = pd.Series(values)
avr_profit1 = values.mean()
avr_profit1

450070252.01413304

Средняя прибыль для региона 1 составила 450 млн. рос. рублей. Ниже считаю доверительный интервал:

In [55]:
lower = values.quantile(0.025)
upper = values.quantile(0.975)
print('2.5% квантиль {: .10}'.format(lower))
print('97.5% квантиль {: .10}'.format(upper))

2.5% квантиль  44596630.49
97.5% квантиль  830796301.5


Диапазон доверительного интервала составил от 45 млн. рос. рублей до 830 млн. рос. рублей. Далее считаем вероятность убытка через метод mean(), примененный к формуле (values < 0):

In [56]:
risks1 = (values < 0).mean()
risks1

0.01

Вероятность убытка составила 1 %.

**Регион 2:**

Считаем значение прибыли для региона 2:

In [57]:
features_2 = region2.drop(['id', 'product'], axis = 1)
target2 = region2['product']
predicted_2 = model_2.predict(features_2)
predicted_2 = pd.Series(predicted_2)

In [58]:
values = []
for i in range(1000):
    target_subsample = target2.sample(n = 500, replace=True, random_state=state)
    probs_subsample = predicted_2[target_subsample.index]
    values.append(profit_calculation(probs_subsample, target_subsample))

values = pd.Series(values)
avr_profit2 = values.mean()
avr_profit2

392181217.03806776

Средняя прибыль для региона 2 составила 392 млн. рос. рублей. Ниже считаю доверительный интервал:

In [59]:
lower = values.quantile(0.025)
upper = values.quantile(0.975)
print('2.5% квантиль {: .10}'.format(lower))
print('97.5% квантиль {: .10}'.format(upper))

2.5% квантиль -145492960.1
97.5% квантиль  876907852.9


Диапазон доверительного интервала составил от '-138' млн. рос. рублей до 884 млн. рос. рублей. Далее считаем вероятность убытка через метод mean(), примененный к формуле (values < 0):

In [60]:
risks2 = (values < 0).mean()
risks2

0.087

Вероятность убытка составила 8,5 %.

Ниже приведен DataFrame common_df со сводной информацией по прибыли и рисками в разбивке регионов:

In [61]:
common_df = pd.DataFrame({'name': ['region_0', 'region_1', 'region_2'], 'profit, ths rub' : [avr_profit0 / 1000, avr_profit1  / 1000, avr_profit2  / 1000],
                          'risk':[risks0, risks1, risks2]})

In [62]:
common_df

,name,"profit, ths rub",risk
0,region_0,431285.313035,0.050
1,region_1,450070.252014,0.010
2,region_2,392181.217038,0.087


**Краткий вывод:**

Наиболшее значение прибыли демонстрирует **Регион 1** (450 млн. рос. сублей против 431 млн. рос. рублей и 392 млн. рос. рублей у прочих регионов), необходимо также отметить, что у данного региона наименьший риск убытков (1 % против 5% и 8,5% у прочих регионов). Соответственно, целесообразно выбрать **Регион 1** для разработки скважин.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован

# Общий вывод

В ходе выполнения самостоятельного проекта для компании «ГлавРосГосНефть» была построена модель машинного обучения, которая определила регион, где добыча принесёт наибольшую прибыль.

Для потроения модели были решены следующие задачи:
1. Были обучены модели для каждого из трех регионов (использован алгоритм LinearRegression);
2. Каждая модель была оценена с помощью метрик регрессии;
3. Был произведен расчет точки безубыточности по объему для одной скважины;
4. Была написана функция для расчета прибыли;
5. С помощью техники Bootstrap была проанализирована возможная прибыль и риски для каждого региона.

По результатам проведения самостоятельного проекта компании «ГлавРосГосНефть» рекомендован **Регион 1** для разработки скважин (он демонстрирует наиболшее значение прибыли и наименьший риск убытков).